# Verifying UK Country Filtering Bug in policyengine.py

This notebook verifies the bug that occurs when filtering simulations by UK country (e.g., Wales).

## The Bug
When running a simulation filtered to a specific UK country (e.g., `region="country/wales"`), we get:
```
ValueError: Unable to set value "[ True  True  True ... False False False]" for variable 
"would_evade_tv_licence_fee", as its length is 8470 while there are 4108 households in the simulation.
```

## Root Cause Hypothesis
The country filtering code in `policyengine/simulation.py` uses DataFrame subsetting:
1. Exports simulation to DataFrame via `to_input_dataframe()`
2. Filters DataFrame rows by country
3. Creates new simulation from filtered DataFrame

The issue is that entity linkage variables (like `household_id`) may not be properly 
exported/imported, causing entity count mismatches during variable calculations.

## Step 1: Setup and Version Check

In [1]:
import numpy as np
import pandas as pd
import traceback

# Import policyengine (the high-level package)
import policyengine
from policyengine import Simulation

# Also import the underlying UK simulation for manual testing
from policyengine_uk import Microsimulation as UKMicrosimulation

## Step 2: Create UK-Wide Baseline Simulation

First, create a standard UK-wide simulation to understand the data structure.

In [2]:
# Create UK-wide simulation using policyengine.Simulation
print("Creating UK-wide simulation...")
print("(This may take a minute to download data)")

sim_uk = Simulation(country="uk", scope="macro")

# Get the underlying microsimulation
underlying_sim = sim_uk.baseline_simulation

print(f"\n=== UK-Wide Simulation Structure ===")
print(f"Person count: {underlying_sim.persons.count}")
print(f"Household count: {underlying_sim.household.count}")
print(f"BenUnit count: {underlying_sim.benunit.count}")

Creating UK-wide simulation...
(This may take a minute to download data)


No data provided, using default dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5
Using dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5



=== UK-Wide Simulation Structure ===
Person count: 115612
Household count: 53508
BenUnit count: 61858


In [3]:
# Create a UK-wide simulation (no region filter)
print("Creating UK-wide simulation...")
sim_uk = Simulation(country="uk", scope="macro")

# Access the underlying country simulation
underlying_sim = sim_uk.baseline_simulation

print(f"\n=== UK-Wide Simulation Structure ===")
print(f"Person count: {underlying_sim.persons.count}")
print(f"Household count: {underlying_sim.household.count}")
print(f"BenUnit count: {underlying_sim.benunit.count}")

Creating UK-wide simulation...


No data provided, using default dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5
Using dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5



=== UK-Wide Simulation Structure ===
Person count: 115612
Household count: 53508
BenUnit count: 61858


## Step 3: Examine to_input_dataframe() Export

This is what `_apply_region_to_simulation` uses to get the data before filtering.

In [4]:
# Export the simulation to DataFrame
print("Exporting simulation to DataFrame...")
df = underlying_sim.to_input_dataframe()

print(f"\n=== Exported DataFrame ===")
print(f"Shape: {df.shape}")
print(f"Number of rows (should be person count): {len(df)}")
print(f"Number of columns: {len(df.columns)}")

Exporting simulation to DataFrame...

=== Exported DataFrame ===
Shape: (115612, 1127)
Number of rows (should be person count): 115612
Number of columns: 1127


In [5]:
# Check for critical entity linkage columns
print("\n=== Critical Entity Columns ===")

critical_patterns = [
    'person_id',
    'household_id', 
    'person_household_id',
    'benunit_id',
    'person_benunit_id'
]

for pattern in critical_patterns:
    matching = [c for c in df.columns if c.startswith(pattern)]
    if matching:
        print(f"  {pattern}: FOUND ({len(matching)} columns)")
        for col in matching[:3]:  # Show first 3
            print(f"    - {col}")
        if len(matching) > 3:
            print(f"    ... and {len(matching) - 3} more")
    else:
        print(f"  {pattern}: MISSING!")


=== Critical Entity Columns ===
  person_id: FOUND (8 columns)
    - person_id__2023
    - person_id__2024
    - person_id__2025
    ... and 5 more
  household_id: FOUND (8 columns)
    - household_id__2023
    - household_id__2024
    - household_id__2025
    ... and 5 more
  person_household_id: FOUND (8 columns)
    - person_household_id__2023
    - person_household_id__2024
    - person_household_id__2025
    ... and 5 more
  benunit_id: FOUND (8 columns)
    - benunit_id__2023
    - benunit_id__2024
    - benunit_id__2025
    ... and 5 more
  person_benunit_id: FOUND (8 columns)
    - person_benunit_id__2023
    - person_benunit_id__2024
    - person_benunit_id__2025
    ... and 5 more


In [6]:
# Check if household_id is exported and examine its values
hh_id_cols = [c for c in df.columns if c.startswith('household_id__')]

print("\n=== household_id Export Analysis ===")
if hh_id_cols:
    col = hh_id_cols[0]
    print(f"Column: {col}")
    print(f"  Length: {len(df[col])}")
    print(f"  Unique values: {df[col].nunique()}")
    print(f"  Min: {df[col].min()}, Max: {df[col].max()}")
    print(f"  Sample values: {df[col].values[:10]}")
else:
    print("household_id NOT exported!")
    print("This could be the root cause of the bug.")


=== household_id Export Analysis ===
Column: household_id__2023
  Length: 115612
  Unique values: 53508
  Min: 1, Max: 67019
  Sample values: [2 1 2 2 2 2 3 6 6 3]


In [7]:
# Check person_household_id linkage
phh_id_cols = [c for c in df.columns if c.startswith('person_household_id__')]

print("\n=== person_household_id Export Analysis ===")
if phh_id_cols:
    col = phh_id_cols[0]
    print(f"Column: {col}")
    print(f"  Length: {len(df[col])}")
    print(f"  Unique values (should match household count): {df[col].nunique()}")
    print(f"  Expected household count: {underlying_sim.household.count}")
    
    if df[col].nunique() == underlying_sim.household.count:
        print("  [OK] Unique count matches household count")
    else:
        print("  [WARNING] Mismatch!")
else:
    print("person_household_id NOT exported! This is critical.")


=== person_household_id Export Analysis ===
Column: person_household_id__2023
  Length: 115612
  Unique values (should match household count): 53508
  Expected household count: 53508
  [OK] Unique count matches household count


## Step 4: Manually Reproduce the Wales Filtering

Let's manually do what `_apply_region_to_simulation` does to identify where it breaks.

In [8]:
# Step 4a: Get country at person level (same as policyengine.py:296-298)
print("=== Step 4a: Calculate country at person level ===")
country_person = underlying_sim.calculate("country", map_to="person").values
print(f"Country array shape: {country_person.shape}")

# Create Wales mask
wales_mask = country_person == "WALES"
print(f"\nWelsh persons: {wales_mask.sum():,}")
print(f"Non-Welsh persons: {(~wales_mask).sum():,}")

=== Step 4a: Calculate country at person level ===
Country array shape: (115612,)

Welsh persons: 8,470
Non-Welsh persons: 107,142


In [9]:
# Step 4b: Filter DataFrame to Wales (same as policyengine.py:299-300)
print("\n=== Step 4b: Filter DataFrame to Wales ===")
df_wales = df[wales_mask]
print(f"Filtered DataFrame shape: {df_wales.shape}")
print(f"Number of Welsh persons: {len(df_wales)}")


=== Step 4b: Filter DataFrame to Wales ===
Filtered DataFrame shape: (8470, 1127)
Number of Welsh persons: 8470


In [10]:
# Check what person_household_id looks like in filtered data
print("\n=== Step 4c: Examine person_household_id in filtered data ===")
if phh_id_cols:
    col = phh_id_cols[0]
    welsh_phh = df_wales[col].values
    print(f"Column: {col}")
    print(f"  Length: {len(welsh_phh)}")
    print(f"  Unique households in Wales: {len(np.unique(welsh_phh))}")
    print(f"  Min household ID: {welsh_phh.min()}")
    print(f"  Max household ID: {welsh_phh.max()}")
    print(f"  Sample values: {welsh_phh[:10]}")
    
    # Check if IDs are contiguous
    unique_hh = np.unique(welsh_phh)
    if np.array_equal(unique_hh, np.arange(len(unique_hh))):
        print("  [INFO] Household IDs are contiguous 0-based")
    else:
        print("  [INFO] Household IDs are NOT contiguous (gaps from filtering)")
        print(f"         This is expected - they're original UK-wide IDs")


=== Step 4c: Examine person_household_id in filtered data ===
Column: person_household_id__2023
  Length: 8470
  Unique households in Wales: 4108
  Min household ID: 2.0
  Max household ID: 66996.0
  Sample values: [2. 2. 2. 2. 2. 6. 6. 6. 6. 7.]
  [INFO] Household IDs are NOT contiguous (gaps from filtering)
         This is expected - they're original UK-wide IDs


## Step 5: Try to Create Simulation from Filtered DataFrame

This is where the error should occur.

In [11]:
# Step 5a: Create new simulation from filtered DataFrame
print("=== Step 5a: Create simulation from filtered DataFrame ===")
print("(This is what policyengine.py:299-300 does)")
print()

try:
    new_sim = UKMicrosimulation(dataset=df_wales)
    
    print(f"New simulation created successfully!")
    print(f"  Person count: {new_sim.persons.count}")
    print(f"  Household count: {new_sim.household.count}")
    print(f"  BenUnit count: {new_sim.benunit.count}")
    
    # Critical check
    if new_sim.household.count == new_sim.persons.count:
        print("\n  [ERROR] Household count equals person count!")
        print("  Entity linkage was lost during filtering.")
        
except Exception as e:
    print(f"[ERROR] Failed to create simulation: {e}")
    traceback.print_exc()

=== Step 5a: Create simulation from filtered DataFrame ===
(This is what policyengine.py:299-300 does)

[ERROR] Failed to create simulation: Unable to set value "[ 39361.   39361.   39361.  ... 134410.5 134410.5   6000. ]" for variable "corporate_wealth", as its length is 8470 while there are 4108 households in the simulation.


Traceback (most recent call last):
  File "/var/folders/qf/7fglpnr94wsc5xgbqkp80xbw0000gn/T/ipykernel_19066/2037714397.py", line 7, in <module>
    new_sim = UKMicrosimulation(dataset=df_wales)
  File "/opt/miniconda3/envs/py-3.13/lib/python3.13/site-packages/policyengine_uk/simulation.py", line 100, in __init__
    self.build_from_dataframe(dataset)
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "/opt/miniconda3/envs/py-3.13/lib/python3.13/site-packages/policyengine_uk/simulation.py", line 286, in build_from_dataframe
    self.set_input(variable, time_period, df[column])
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/py-3.13/lib/python3.13/site-packages/policyengine_core/simulations/simulation.py", line 1241, in set_input
    self.get_holder(variable_name).set_input(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        period, value, self.branch_name
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/opt/miniconda3/envs/py-3.13/lib/python3

In [12]:
# Step 5b: Check if household_id holder has data
print("\n=== Step 5b: Check household_id holder ===")

try:
    hh_id_holder = new_sim.get_holder("household_id")
    known_periods = list(hh_id_holder.get_known_periods())
    print(f"household_id known periods: {known_periods}")
    
    if known_periods:
        period = known_periods[0]
        arr = hh_id_holder.get_array(period)
        print(f"  Period {period}: array shape = {arr.shape if arr is not None else 'None'}")
        if arr is not None:
            print(f"  Values sample: {arr[:10]}")
    else:
        print("  No known periods - household_id was not set as input!")
except Exception as e:
    print(f"Error checking household_id: {e}")


=== Step 5b: Check household_id holder ===
Error checking household_id: name 'new_sim' is not defined


In [13]:
# Step 5c: Try to calculate household_id
print("\n=== Step 5c: Calculate household_id ===")

try:
    hh_ids = new_sim.calculate("household_id", 2025)
    print(f"household_id calculation result:")
    print(f"  Length: {len(hh_ids)}")
    print(f"  Expected (household count): {new_sim.household.count}")
    
    if len(hh_ids) == new_sim.household.count:
        print("  [OK] Length matches household count")
    else:
        print(f"  [ERROR] Length mismatch! Got {len(hh_ids)}, expected {new_sim.household.count}")
        
except Exception as e:
    print(f"Error calculating household_id: {e}")
    traceback.print_exc()


=== Step 5c: Calculate household_id ===
Error calculating household_id: name 'new_sim' is not defined


Traceback (most recent call last):
  File "/var/folders/qf/7fglpnr94wsc5xgbqkp80xbw0000gn/T/ipykernel_19066/1284064109.py", line 5, in <module>
    hh_ids = new_sim.calculate("household_id", 2025)
             ^^^^^^^
NameError: name 'new_sim' is not defined


## Step 6: Try to Calculate would_evade_tv_licence_fee

This is the variable that triggers the error in production.

In [14]:
# Step 6: Calculate the problematic variable
print("=== Step 6: Calculate would_evade_tv_licence_fee ===")
print("(This calculation uses random(household) internally)")
print()

try:
    result = new_sim.calculate("would_evade_tv_licence_fee", 2025)
    print(f"Calculation succeeded!")
    print(f"  Result length: {len(result)}")
    print(f"  Expected (household count): {new_sim.household.count}")
    print(f"  Result dtype: {result.dtype}")
    
except ValueError as e:
    print(f"[EXPECTED ERROR] ValueError:")
    print(f"  {e}")
    print()
    print("This confirms the bug!")
    
    # Parse the error message
    error_str = str(e)
    if "length is" in error_str and "while there are" in error_str:
        print("\nThe error indicates:")
        print("  - The formula returned an array sized for persons")
        print("  - But the variable is household-level")
        print("  - This means random(household) returned wrong-sized array")
        
except Exception as e:
    print(f"Unexpected error: {type(e).__name__}: {e}")
    traceback.print_exc()

=== Step 6: Calculate would_evade_tv_licence_fee ===
(This calculation uses random(household) internally)

Unexpected error: NameError: name 'new_sim' is not defined


Traceback (most recent call last):
  File "/var/folders/qf/7fglpnr94wsc5xgbqkp80xbw0000gn/T/ipykernel_19066/1304269510.py", line 7, in <module>
    result = new_sim.calculate("would_evade_tv_licence_fee", 2025)
             ^^^^^^^
NameError: name 'new_sim' is not defined


## Step 7: Test Using policyengine.Simulation Directly

Now let's test using the high-level API to confirm the bug occurs there too.

In [15]:
# Test with policyengine.Simulation using region="country/wales"
print("=== Step 7: Test with policyengine.Simulation ===")
print("Creating Simulation with region='country/wales'...")
print()

try:
    sim_wales = Simulation(country="uk", scope="macro", region="country/wales")
    
    wales_underlying = sim_wales.baseline_simulation
    print(f"Wales simulation created!")
    print(f"  Person count: {wales_underlying.persons.count}")
    print(f"  Household count: {wales_underlying.household.count}")
    
    # Try calculating the problematic variable
    print("\nCalculating would_evade_tv_licence_fee...")
    result = sim_wales.calculate("would_evade_tv_licence_fee")
    print(f"  Result length: {len(result)}")
    print("  [OK] No error!")
    
except Exception as e:
    print(f"[ERROR] {type(e).__name__}: {e}")
    print()
    print("This confirms the bug exists in the high-level API.")
    traceback.print_exc()

=== Step 7: Test with policyengine.Simulation ===
Creating Simulation with region='country/wales'...



No data provided, using default dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5
Using dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5


DataFrame columns: ['miscellaneous_income__2023', 'miscellaneous_income__2024', 'miscellaneous_income__2025', 'miscellaneous_income__2026', 'miscellaneous_income__2027', 'miscellaneous_income__2028', 'miscellaneous_income__2029', 'miscellaneous_income__2030', 'corporate_wealth__2023', 'corporate_wealth__2024', 'corporate_wealth__2025', 'corporate_wealth__2026', 'corporate_wealth__2027', 'corporate_wealth__2028', 'corporate_wealth__2029', 'corporate_wealth__2030', 'non_residential_property_value__2023', 'non_residential_property_value__2024', 'non_residential_property_value__2025', 'non_residential_property_value__2026', 'non_residential_property_value__2027', 'non_residential_property_value__2028', 'non_residential_property_value__2029', 'non_residential_property_value__2030', 'employment_income_before_lsr__2023', 'employment_income_before_lsr__2024', 'employment_income_before_lsr__2025', 'employment_income_before_lsr__2026', 'employment_income_before_lsr__2027', 'employment_income_bef

Traceback (most recent call last):
  File "/var/folders/qf/7fglpnr94wsc5xgbqkp80xbw0000gn/T/ipykernel_19066/3661659745.py", line 7, in <module>
    sim_wales = Simulation(country="uk", scope="macro", region="country/wales")
  File "/Users/administrator/Documents/PolicyEngine/policyengine.py/policyengine/simulation.py", line 110, in __init__
    self._initialise_simulations()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/administrator/Documents/PolicyEngine/policyengine.py/policyengine/simulation.py", line 202, in _initialise_simulations
    self.baseline_simulation = self._initialise_simulation(
                               ~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        scope=self.options.scope,
        ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
        subsample=self.options.subsample,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/administrator/Documents/PolicyEngine/policyengine.py/policyengine/simulation.py", line 260, in _initialise_simulation
    simulation =

## Step 8: Compare with Constituency Filtering (Should Work)

Constituency filtering uses weight adjustment instead of DataFrame subsetting.

In [16]:
# Test constituency filtering
print("=== Step 8: Test Constituency Filtering ===")
print("Creating Simulation with region='constituency/Cardiff South and Penarth'...")
print()

try:
    sim_const = Simulation(
        country="uk", 
        scope="macro", 
        region="constituency/Cardiff South and Penarth"
    )
    
    const_underlying = sim_const.baseline_simulation
    print(f"Constituency simulation created!")
    print(f"  Person count: {const_underlying.persons.count}")
    print(f"  Household count: {const_underlying.household.count}")
    print("  (Full UK counts, but weights adjusted for constituency)")
    
    # Try calculating the problematic variable
    print("\nCalculating would_evade_tv_licence_fee...")
    result = sim_const.calculate("would_evade_tv_licence_fee")
    print(f"  Result length: {len(result)}")
    print("  [OK] Constituency filtering works!")
    
except Exception as e:
    print(f"[ERROR] {type(e).__name__}: {e}")
    traceback.print_exc()

No data provided, using default dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5
Using dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5


=== Step 8: Test Constituency Filtering ===
Creating Simulation with region='constituency/Cardiff South and Penarth'...



Constituency simulation created!
  Person count: 115612
  Household count: 53508
  (Full UK counts, but weights adjusted for constituency)

Calculating would_evade_tv_licence_fee...
[ERROR] AttributeError: 'Simulation' object has no attribute 'calculate'


Traceback (most recent call last):
  File "/var/folders/qf/7fglpnr94wsc5xgbqkp80xbw0000gn/T/ipykernel_19066/2462177757.py", line 21, in <module>
    result = sim_const.calculate("would_evade_tv_licence_fee")
             ^^^^^^^^^^^^^^^^^^^
AttributeError: 'Simulation' object has no attribute 'calculate'


In [17]:
# Test local authority filtering
print("\n=== Step 8b: Test Local Authority Filtering ===")
print("Creating Simulation with region='local_authority/Cardiff'...")
print()

try:
    sim_la = Simulation(
        country="uk", 
        scope="macro", 
        region="local_authority/Cardiff"
    )
    
    la_underlying = sim_la.baseline_simulation
    print(f"Local Authority simulation created!")
    print(f"  Person count: {la_underlying.persons.count}")
    print(f"  Household count: {la_underlying.household.count}")
    print("  (Full UK counts, but weights adjusted for LA)")
    
    # Try calculating the problematic variable
    print("\nCalculating would_evade_tv_licence_fee...")
    result = sim_la.calculate("would_evade_tv_licence_fee")
    print(f"  Result length: {len(result)}")
    print("  [OK] Local authority filtering works!")
    
except Exception as e:
    print(f"[ERROR] {type(e).__name__}: {e}")
    traceback.print_exc()


=== Step 8b: Test Local Authority Filtering ===
Creating Simulation with region='local_authority/Cardiff'...



No data provided, using default dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5
Using dataset: gs://policyengine-uk-data-private/enhanced_frs_2023_24.h5


KeyboardInterrupt: 

## Step 9: Deep Dive - Check random() Function Behavior

In [ ]:
# Check what random(household) would return in the broken simulation
print("=== Step 9: Investigate random() function behavior ===")

# Import the random function
from policyengine_core.commons.formulas import random

try:
    # Get household population from the new (potentially broken) simulation
    hh_pop = new_sim.household
    print(f"Household population count: {hh_pop.count}")
    
    # Check what household_id returns when calculated via population
    print("\nCalling hh_pop('household_id', 2025)...")
    hh_ids_from_pop = hh_pop("household_id", 2025)
    print(f"  Result length: {len(hh_ids_from_pop)}")
    print(f"  Expected: {hh_pop.count}")
    
    if len(hh_ids_from_pop) != hh_pop.count:
        print(f"\n  [BUG CONFIRMED] household_id returned {len(hh_ids_from_pop)} values")
        print(f"  but household population only has {hh_pop.count} entities!")
        print("  This is why random(household) fails.")
        
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Summary and Conclusions

In [ ]:
print("="*70)
print("DIAGNOSTIC SUMMARY")
print("="*70)

print("""
FINDINGS:

1. COUNTRY FILTERING (country/wales):
   - Uses to_input_dataframe() + DataFrame subsetting + new Simulation()
   - Creates entity count mismatch between persons and households
   - Breaks when calculating variables that use random(household)

2. CONSTITUENCY/LA FILTERING:
   - Uses weight adjustment on existing simulation
   - Preserves entity structure
   - Works correctly

ROOT CAUSE:
   - The to_input_dataframe() -> filter -> new Simulation() approach
     doesn't properly preserve entity relationships
   - Either household_id isn't properly exported/imported, OR
   - The entity membership mapping gets corrupted during rebuild

RECOMMENDED FIX:
   - Use weight-based filtering for country filtering (like constituency/LA)
   - Zero out weights for households not in the target country
   - This preserves entity structure and avoids the export/import complexity

Example fix for policyengine/simulation.py:

    if "country/" in region:
        country_name = region.split("/")[1]
        country = simulation.calculate("country", map_to="household").values
        is_in_country = (country == country_name.upper())
        current_weights = simulation.calculate(
            "household_weight", simulation.default_calculation_period
        )
        simulation.set_input(
            "household_weight",
            simulation.default_calculation_period,
            current_weights * is_in_country  # Zero out non-matching
        )
""")